Setup library

In [1]:
import duckdb

duckdb Setup

In [2]:
con = duckdb.connect()
con.sql("INSTALL httpfs; LOAD httpfs;")

Setup File Path & Object Name di MinIO

In [3]:
sourceFilePath = {
    'bekasi': r'C:\Users\Asus\Downloads\Data Andalan Utama (Intern)\Exercise_Data_Sekolah_2\matchData\revisi\part2\hasilAkhir\Bekasi.csv',
    'depok': r'C:/Users/Asus/Downloads/Data Andalan Utama (Intern)/Exercise_Data_Sekolah_2/matchData/revisi/part2/hasilAkhir/Depok.csv'
    }

targetFilePath = {
    'bekasi': '../parquetStaging/bekasi.parquet',
    'depok': '../parquetStaging/depok.parquet'
}
    
minioObjectName = {
    'bekasi': 'JawaBarat/dataBekasi.parquet',
    'depok': 'JawaBarat/dataDepok.parquet'
    }

In [4]:
# bekasi csv to parquet
bekasiPqrquet = con.execute(f"""
            COPY             
            (SELECT * FROM read_csv_auto('{sourceFilePath['bekasi']}', AUTO_DETECT=TRUE))

            TO
            '{targetFilePath['bekasi']}' (FORMAT 'PARQUET');
            """)

In [5]:
# depok csv to parquet
depokParquet = con.execute(f"""
            COPY             
            (SELECT * FROM read_csv_auto('{sourceFilePath['depok']}', AUTO_DETECT=TRUE))

            TO
            '{targetFilePath['depok']}' (FORMAT 'PARQUET');
            """)

MinIO Setup

In [6]:
minioAuth = {
    'accessKey': 'minioadmin',
    'secretKey': 'miniopassword',
    'bucket': 'data-dapodik-kemendikdasmen',
    'endpoint': 'http://localhost:9000'
}

Minio Connection

In [7]:
con.sql(f"SET s3_endpoint = '{minioAuth['endpoint'].replace('http://', '')}';")
con.sql("SET s3_use_ssl = false;") 
con.sql(f"SET s3_access_key_id = '{minioAuth['accessKey']}';")
con.sql(f"SET s3_secret_access_key = '{minioAuth['secretKey']}';")
con.sql("SET s3_url_style = 'path';") # WAJIB untuk MinIO/Local

Full EndPoint URL

In [8]:
fullEndPointURLBekasi = f"s3://{minioAuth['bucket']}/{minioObjectName['bekasi']}"
fullEndPointURLDepok = f"s3://{minioAuth['bucket']}/{minioObjectName['depok']}"

In [9]:
loadBekasi = f"""
    COPY 
        (
            SELECT *
            FROM 
                read_parquet('{targetFilePath['bekasi']}')
        ) 
    TO 
        '{fullEndPointURLBekasi}'
    (FORMAT 'PARQUET', 
     OVERWRITE TRUE);
"""

loadDepok = f"""
    COPY 
        (
            SELECT *
            FROM 
                read_parquet('{targetFilePath['depok']}')
        ) 
    TO 
        '{fullEndPointURLDepok}'
    (FORMAT 'PARQUET', 
     OVERWRITE TRUE);
"""

In [10]:
# --- 2. Jalankan Proses Migrasi ---
try:
    con.sql(loadBekasi)
    con.sql(loadDepok)
    print(f"\n✅ Sukses! Data CSV telah dibaca, dikonversi ke Parquet, dan dikirim langsung ke MinIO")
except Exception as e:
    print(f"\n❌ Terjadi error saat migrasi: {e}")


✅ Sukses! Data CSV telah dibaca, dikonversi ke Parquet, dan dikirim langsung ke MinIO


In [11]:
con.close()